Yes we can
----

In [ ]:
import time
start = time.clock()

#open data
import codecs
import nltk #language functions
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

datas = pd.read_csv('../input/train.csv')
datas = datas.fillna('leeg')
#print(datas.head())

def cleantxt(x):    # aangeven sentence
    x = x.lower()
    # Pad punctuation with spaces on both sides
    for char in ['.', '"', ',', '(', ')', '!', '?', ';', ':']:
        x = x.replace(char, ' ' + char + ' ')
    return x.split()

datas['question1']=datas['question1'].map(cleantxt)
datas['question2']=datas['question2'].map(cleantxt)

print(datas.head())
end = time.clock()
print('open:',end-start)

#datas

Sentences where the only difference are stopwords are imho to be considered as similar.
----

you can discuss about this, but when the question has only the stopwords being different, and as everybody is throwing away the stopwords, you endup with a signal loss in your data...if you think those questions are not similar.
You can't even train on it, since you throw away that information you endup with nois imho ?


In [ ]:
 # Import the stop word list
stoppers=stopwords.words("english") 
def falsdupl(sent1, sent2,isdup):
    difq1 = [w for w in sent2 if w not in sent1 if not w in stoppers]
    difq2 = [w for w in sent1 if w not in sent2 if not w in stoppers]
    if len(difq1+difq2)==0:
        falsdup=1
    else:
        falsdup=isdup
        
    return falsdup

start = time.clock()
datas['fals_dup']=datas[['question1','question2','is_duplicate']].apply(lambda x: falsdupl(*x), axis=1)

    
end = time.clock()
print('all dubious:',400000/(end-start))
#amazing four times faster this way...

Find Is_duplicate,is_notduplicate and possible false not duplicate
---

 - we will try to plot the ration duplicate / nr of questions  progressively in the database. 
 - the reverse function is the 'non duplicate progressively
 - the final one is the most interesting: the ratio fals duplicate / nr of labeled as duplicate.

I suspect a rampup to the end.




In [ ]:
tel_nr_pos = 0
tel_nr_qu = len(datas)
tel_nr_neg = 0
tel_nr_fals = 0
reeks = pd.DataFrame(columns=('pos','neg','tel','fals'))
print(reeks)
for xyz in range(0,len(datas)):
    #q1=datas.iloc[xyz].question1
    #q2=datas.iloc[xyz].question2
    #sent1=q1.split()
    #sent2=q2.split()
    #equq1 = [w for w in sent1 if w in sent2]
    #difq1 = [w for w in sent1 if w not in sent2]
    #difq2 = [w for w in sent2 if w not in sent1]
    #diftot = difq1+difq2
    #difton = [w for w in diftot if not w in stopwords.words("english")]
    dupmem=datas.at[xyz,'is_duplicate']
    if dupmem==0:
        tel_nr_neg+=1
    if dupmem==1:
        tel_nr_pos+=1
    if dupmem<datas.at[xyz,'fals_dup']:
        tel_nr_fals+=1
    if xyz/100==round(xyz/100) and xyz>1:
        reeks.loc[xyz/100]= [ tel_nr_pos/xyz,tel_nr_neg/xyz,xyz,tel_nr_fals/tel_nr_pos ]
        end = time.clock()
        if end-start>1000:
            break
        
print(reeks)
end = time.clock()
print('graphing:',end-start)

import matplotlib.pyplot as plt  
from pandas.tools.plotting import scatter_matrix
scatter_matrix(reeks, alpha=0.2, figsize=(6, 6), diagonal='kde')

The graphs shows a stable descend in positives, stable rise negatives 
----

 - 37% duplicate in training set
 - a negative correlation between pos/neg meaning the ratio descends
 - 2.8% false positives



In [ ]:
# a 37% positive duplicates diminishing to the end
# a 4% what i call false negative, or negative with only stopwords